# Задание 2.2 Кодирование марковского источника.
  
Для декапитализированного текста построить все контекстные модели 3-го порядка   
(для каждого контекста собрать всю информацию, как в лекции 7).  
Посчитать необходимый объём памяти для хранения всех моделей в несжатом виде.   
Выбрать и обосновать способ кодирования для передачи модели декодеру,  
указать длину получившегося кода и дать ссылку на использовавшиеся скрипты.  

Посчитаем для каждой подстроки длины 3 частоты её правого контекста длины 1 

In [1]:
from collections import defaultdict

with open('decapitalized.txt', 'r', encoding='ASCII') as f:
    text = f.read()


max_order = 3
counts = defaultdict(lambda: defaultdict(int))

for i in range(max_order, len(text)):
    context = text[i - max_order : i]  # берём 3 символа перед text[i]
    next_char = text[i]
    counts[context][next_char] += 1


for test_context in ['imm', 'kan', 'rea', 'pur']:
    print(f'{test_context} : {dict(counts[test_context])}')

imm : {'a': 22, 'o': 14, 'e': 99, 'i': 1, 'u': 1}
kan : {'t': 1}
rea : {'s': 1339, 't': 230, 'l': 545, 'd': 106, 'c': 40, 'f': 1, 'm': 10, 'k': 9, 'p': 3, 'n': 2, 'r': 2}
pur : {'e': 807, 'p': 142, 's': 43, 'i': 13, 'a': 1}


Посчитаем необходимый объём памяти для хранения всех моделей в несжатом виде.  
Для каждой модели нужно хранить:  
1. саму модель s (3 байта)  
2. число $n_s$ символов, встречавшихся в контексте  
3. сами встречавшиеся символы (по байту на символ)  
4. частоты встречаемости символов в контексте ($n_s$ чисел)  

In [38]:
def count_memory_size(models):
    """ Вычисляет объем памяти, необходимый для хранения моделей """
    total_bytes = 0
    for context, symbols in models.items():
        # 1. Сам контекст s (3 байта)
        total_bytes += 3
        # 2. Число символов n_s (1 байт)
        n_s = len(symbols)
        total_bytes += 1
        # 3. Сами символы (n_s байт)
        total_bytes += n_s
        # 4. Частоты встречаемости символов (n_s чисел по 4 байта каждое)
        total_bytes += n_s * 4

    return total_bytes  

In [39]:
memory_size = count_memory_size(counts)

print(f'Для хранения всех моделей в несжатом виде требуется {memory_size} байт или {round(memory_size / 1024, 2)} кб')

Для хранения всех моделей в несжатом виде требуется 167050 байт или 163.13 кб


In [40]:
def encode_to_unified_raw(counts):
    unified_raw = ''
    for context, symbols in counts.items():
        # Добавляем контекст и число символов
        unified_raw += context
        unified_raw += str(len(symbols))
        # Добавляем пары (символ, частота)
        for char, count in symbols.items():
            unified_raw  += char
            unified_raw += str(count)
    return unified_raw

In [41]:
unified_raw = encode_to_unified_raw(counts)
print(unified_raw[:20])

   28 4633110t51b7p3


Попробуем сжать данную последовательность

Для сжатия возьмём lz77, реализованный в самом Питоне

In [77]:
import zlib

# Преобразуем все элементы в строки и объединяем
bytes_data = unified_raw.encode('ASCII')
bytes_data[:100]

b'   28 4633110t51b7p33m4n11q6i24d4h2"1f3s53a32c25j1u52939r748o49l1e10g4w6-1  1571.15\n5 2s1 17384\n1)11'

In [78]:
compressed = zlib.compress(bytes_data, level=8)

print(f'Для хранения моделей в сжатом виде необходимо {round(len(compressed) / 1024, 2)} кб')

Для хранения моделей в сжатом виде необходимо 59.41 кб


In [64]:
# 1. Декомпрессия
decompressed_data = zlib.decompress(compressed)

# 2. Декодирование в строку
restored_str = decompressed_data.decode('ASCII')
restored_str[:50]

'   28 4633110t51b7p33m4n11q6i24d4h2"1f3s53a32c25j1'

## Представленные данные можно разбить на численные и буквенные последовательности, имея которые можно восстановить исходный марковский источник

In [44]:
def split_model(counts):
    char_sequence = []
    num_sequence = []
    for context, symbols in counts.items():
        # 1. Добавляем контекст и символы в буквенную часть
        char_sequence.append(context)
        char_sequence.extend(symbols.keys())
        # 2. Добавляем n_s и частоты в числовую часть
        num_sequence.append(len(symbols))
        num_sequence.extend(symbols.values())
    return char_sequence, num_sequence

char_sequence, num_sequence = split_model(counts)

print(char_sequence[:10])
print(num_sequence[:10])

['   ', ' ', '1', 't', 'b', 'p', 'm', 'n', 'q', 'i']
[28, 4633, 10, 51, 7, 33, 4, 11, 6, 24]


In [45]:
print(len(char_sequence))
print(len(num_sequence))

34786
34786


### Сожмём числовую последовательность

Посмотрим на распределение чисел в num_sequence

In [10]:
print(f'Максимальное число = {max(num_sequence)}')
print(f'Минимальное число = {min(num_sequence)}')

counter = 0
for i in num_sequence:
    if i <= 254:
        counter += 1

print(f'Процент чисел, меньших 254 = {round(counter / len(num_sequence) * 100, 2)}%')

Максимальное число = 18302
Минимальное число = 1
Процент чисел, меньших 254 = 96.98%


#### Кодирование с переполнением

То есть нам необходимо кодировать в основном маленькие числа из небольшого диапазона, для чего кодирование с переполнением отлично подходит.  
Реализуем его по схеме из ДЗ 1.4 (<= 254 - 8 бит, >= 255 - 24 бита)

In [11]:
def encode_number(num):
    result_bits = []

    if num <= 254:
        # Просто 8-битное представление
        bits = f"{num:08b}"
        result_bits.append(bits)
    else:
        # 11111111 префикс
        result_bits.append("11111111")
        
        value_to_encode = num - 255
        
        # Определяем число бит N в двоичном представлении числа 
        N = value_to_encode.bit_length()
        
        leading_zeros = 16 - N # количество ведущих нулей
        
        # Добавляем leading zeros
        result_bits.append("0" * leading_zeros)
        
        # Добавляем N битов (если N > 0)
        if N > 0:
            bits = f"{value_to_encode:0{N}b}"
            result_bits.append(bits)
    
    return ''.join(result_bits)


encoded_num_sequence = ''

for num in num_sequence:
    encoded_num_sequence += encode_number(num)

In [12]:
num = 18302
print(f'Длина кода фиксированной длины = {round(num.bit_length() * len(num_sequence) / 8 / 1024, 2)} кб')

Длина кода фиксированной длины = 63.7 кб


In [13]:
num_sequence_mem_size = round(len(encoded_num_sequence) / 8 / 1024, 2)
print(f'Длина кода с переполнением = {num_sequence_mem_size} кб')

Длина кода с переполнением = 36.02 кб


Можно заметить, что доля чисел <= 14 в последовательности = 70%. Поэтому попробуем другое деление:  
4 бит для X <= 14  
12 бит для 269 >= X >= 15  
24 бит для X >= 270  

In [14]:
def encode_number(x):
    result_bits = []

    if x <= 14:
        # 4-битное представление числа
        bits = f"{x:04b}"
        result_bits.append(bits)

    elif 15 <= x <= 269:
        result_bits.append("1111")  # 4 бита

        value_to_encode = x - 15

        N = value_to_encode.bit_length()

        leading_zeros = 8 - N

        # Добавляем ведущие нули
        result_bits.append("0" * leading_zeros)

        # Добавляем N бит (если N > 0)
        if N > 0:
            bits = f"{value_to_encode:0{N}b}"
            result_bits.append(bits)

    elif x >= 270:
        result_bits.append("11111111")  # 8 бит

        value_to_encode = x - 255

        N = value_to_encode.bit_length()

        leading_zeros = 16 - N

        result_bits.append("0" * leading_zeros)

        if N > 0:
            bits = f"{value_to_encode:0{N}b}"
            result_bits.append(bits)

    else:
        raise ValueError("Unsupported value X")  # на всякий случай защита

    return ''.join(result_bits)


encoded_num_sequence_ver_2 = ''

for num in num_sequence:
    encoded_num_sequence_ver_2 += encode_number(num)

print(num_sequence[0])
encoded_num_sequence_ver_2[:12]

28


'111100001101'

In [15]:
num_sequence_mem_size = round(len(encoded_num_sequence_ver_2) / 8 / 1024, 2)
print(f'Длина кода с переполнением = {num_sequence_mem_size} кб')

Длина кода с переполнением = 26.55 кб


#### Кодирование дельта кодом Элиаса

Реализация метода была взята отсюда https://github.com/keon/algorithms/blob/master/algorithms/compression/elias.py

In [20]:
import elias

delta_compressed_list = [elias.elias_delta(x) for x in num_sequence]

delta_compressed_list[:5]

['110011100', '1110101001000011001', '11000010', '1101010011', '10111']

In [22]:
num_sequence_mem_size_delta = round(len(''.join(delta_compressed_list)) / 8 / 1024, 2)
print(f'Длина Дельта кода Элиаса = {num_sequence_mem_size_delta} кб')

Длина Дельта кода Элиаса = 25.18 кб


#### Кодирование кодами Фибоначчи

In [28]:
def fibonacci_coding(n):
    if n == 0:
        return "11"
    
    # Генерируем числа Фибоначчи до N
    fib_sequence = [1, 2]
    while fib_sequence[-1] <= n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    fib_sequence = fib_sequence[:-1]  # Удаляем последний элемент
    
    code = ['0'] * (len(fib_sequence))
    remainder = n
    
    # Проходим по числам Фибоначчи от большего к меньшему
    for i in range(len(fib_sequence)-1, -1, -1):
        if fib_sequence[i] <= remainder:
            remainder -= fib_sequence[i]
            code[i] = '1'
    
    # Добавляем завершающую 1
    code.append('1')
    
    return ''.join(code)

In [29]:
fib_compressed_list = [fibonacci_coding(x) for x in num_sequence]

fib_compressed_list[:5]

['01010011', '0101010010001000011', '010011', '101001011', '01011']

In [55]:
num_sequence_mem_size_fib = len(''.join(fib_compressed_list))
print(f'Длина кода Фибоначчи = {round(num_sequence_mem_size_fib / 8 / 1024, 2)} кб')

Длина кода Фибоначчи = 21.13 кб


Код Фибоначчи победил

### Сожмём последовательность символьную

In [74]:
# Преобразуем все элементы в строки и объединяем в одну строку
bytes_data_char_sequence = ''.join(char_sequence).encode('ASCII')

bytes_data_char_sequence[:100]

b'    1tbpmnqidh"fsacju23r4olegw-  17.\n s 178\n)17817781\n81\n\n1\n\n \n\n  \n   hit*afwsnmbkeojv123ry4g(5dcplu'

In [57]:
compressed_char_sequence = zlib.compress(bytes_data_char_sequence, level=8)

print(f'Для хранения буквенной последовательности в сжатом виде необходимо {round(len(compressed_char_sequence) / 1024, 2)} кб')

Для хранения буквенной последовательности в сжатом виде необходимо 29.36 кб


In [59]:
full_mem_size = round((len(compressed_char_sequence) + num_sequence_mem_size_fib / 8) / 1024 , 2)
print(f'В случае разбиения на числовую и буквенную последовательности необходимо памяти {full_mem_size} кб')

print(f'Что позволило сэкономить {round(len(compressed) / 1024 - full_mem_size, 2)} кб')

В случае разбиения на числовую и буквенную последовательности необходимо памяти 50.49 кб
Что позволило сэкономить 8.92 кб


## Теперь попробуем сжать с помощью PPM, взятого из https://github.com/miurahr/pyppmd

In [28]:
!pip install pyppmd

In [60]:
import pyppmd

In [65]:
compressed_by_PPM = pyppmd.compress(bytes_data)

print(f'Для хранения моделей в сжатом PPM виде необходимо {round(len(compressed_by_PPM) / 1024, 2)} кб')
print(f'В случае LZ77 было необходимо {round(len(compressed) / 1024, 2)} кб')

Для хранения моделей в сжатом PPM виде необходимо 54.32 кб
В случае LZ77 было необходимо 59.41 кб


In [66]:
PPM_compressed_char_sequence = pyppmd.compress(bytes_data_char_sequence)

print(f'Для хранения буквенной последовательности в сжатом виде необходимо {round(len(PPM_compressed_char_sequence) / 1024, 2)} кб')

Для хранения буквенной последовательности в сжатом виде необходимо 28.61 кб


In [67]:
full_mem_size = round((len(PPM_compressed_char_sequence) + num_sequence_mem_size_fib / 8) / 1024 , 2)
print(f'В случае разбиения на числовую и буквенную последовательности необходимо памяти {full_mem_size} кб')

print(f'Что позволило сэкономить {round(len(compressed_by_PPM) / 1024 - full_mem_size, 2)} кб')

В случае разбиения на числовую и буквенную последовательности необходимо памяти 49.74 кб
Что позволило сэкономить 4.58 кб


То есть простое сжатие без разбиения на числовую и буквенную последовательности в случае PPM метода не улучшило степень сжатия.  
Возможная причина - в буквенную последовательность все равно входят не только буквы, но и цифры и другие символы

In [ ]:
compression_ratio = (len(PPM_compressed_char_sequence) + num_sequence_mem_size_fib / 8) / memory_size
print(f'Полученная степень сжатия марковского источника с помощью кодирования Фибоначчи и PPM = {round(compression_ratio, 5)}')

Полученная степень сжатия марковского источника с помощью кодирования Фибоначчи и PPM = 0.30489


(Степень сжатия улучшилась примерно на 0.03 благодаря улучшению сжатия чисел)